<a href="https://colab.research.google.com/github/reyhanfisena/chatbot-using-cosine-similarity/blob/main/Cosine_Similarity_based_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Library

In [6]:
import numpy as np
import pandas as pd
import string
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Download stopwords jika belum ada
nltk.download('stopwords')
from nltk.corpus import stopwords

from google.colab import drive
drive.mount('/content/drive')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Mounted at /content/drive


##Load Dataset

In [7]:
def load_dataset(file_path):
    questions = []
    answers = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            if '?' in line:
                q, a = line.strip().split('?', 1)
                questions.append(q + '?')
                answers.append(a.strip())
    return questions, answers

# Load dataset
file_path = 'drive/MyDrive/data_kesehatan.txt'
questions, answers = load_dataset(file_path)

# Tampilkan beberapa contoh data
print("Contoh Pertanyaan:", questions)
print("Contoh Jawaban:", answers)

Contoh Pertanyaan: ['Apa itu pola makan sehat?', 'Bagaimana cara mengurangi berat badan dengan aman?', 'Apa saja gejala umum dari tekanan darah tinggi?', 'Apa manfaat dari berolahraga setiap hari?', 'Bagaimana cara menjaga kesehatan mental?', 'Apa penyebab utama diabetes tipe 2?', 'Bagaimana cara menjaga kesehatan jantung?', 'Berapa lama waktu tidur yang ideal bagi orang dewasa?', 'Apa saja makanan yang baik untuk kesehatan otak?', 'Apakah penting untuk mengonsumsi cukup air setiap hari?', 'Apa yang dimaksud dengan detoksifikasi tubuh?', 'Bagaimana cara mengurangi stres?', 'Mengapa penting untuk memiliki pola makan yang teratur?', 'Apa yang harus dilakukan saat terkena flu?', 'Bagaimana cara menghindari osteoporosis?', 'Apa saja tips untuk memiliki kulit sehat?', 'Bagaimana cara menambah berat badan secara sehat?', 'Apa itu diet ketogenik?', 'Bagaimana cara meningkatkan sistem kekebalan tubuh?', 'Apa yang harus dilakukan untuk menjaga kesehatan mata?']
Contoh Jawaban: ['Pola makan seha

## Preprocessing

In [8]:
def preprocess(text):
    # Ubah teks ke huruf kecil
    text = text.lower()
    # Hapus tanda baca
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenisasi dan hapus stopwords
    tokens = text.split()
    tokens = [word for word in tokens if word not in stopwords.words('indonesian')]
    return ' '.join(tokens)

# Preprocess semua pertanyaan dalam dataset
preprocessed_questions = [preprocess(q) for q in questions]

## Responses Chatbot

In [12]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(preprocessed_questions)

def get_response(user_input):
    # Preprocessing input pengguna
    preprocessed_input = preprocess(user_input)
    # Transform input ke dalam bentuk TF-IDF
    input_vec = vectorizer.transform([preprocessed_input])
    # Hitung Cosine Similarity
    cosine_sim = cosine_similarity(input_vec, tfidf_matrix)
    # Cari indeks dari jawaban dengan kesamaan tertinggi
    index = cosine_sim.argsort()[0][-1]
    similarity = cosine_sim[0][index]
    # Tetapkan threshold similarity untuk memastikan relevansi jawaban
    if similarity < 0.1:
        return "Maaf, saya tidak mengerti pertanyaan Anda."
    else:
        return answers[index]

## Test Chatbot

In [13]:
def chat():
    print("Halo, saya Chatbot-health. Apa pertanyaanmu terkait kesehatan? Ketik 'exit' untuk keluar.")
    while True:
        user_input = input("Anda: ")
        if user_input.lower() == 'exit':
            print("Chatbot-health: Terimakasih, Sampai jumpa!")
            break
        response = get_response(user_input)
        print(f"Chatbot-health: {response}")

# Mulai percakapan dengan chatbot
chat()

Halo, saya Chatbot-health. Apa pertanyaanmu terkait kesehatan? Ketik 'exit' untuk keluar.
Anda: apa pola makan sehat itu?
Chatbot-health: Pola makan sehat adalah pola makan yang mengandung nutrisi seimbang untuk menjaga kesehatan tubuh.
Anda: apakah kita harus minum air cukup setiap hari?
Chatbot-health: Ya, penting untuk minum cukup air setiap hari untuk menjaga fungsi tubuh, menghindari dehidrasi, dan membantu metabolisme.
Anda: mengapa penting untuk menjaga pola makan sehat?
Chatbot-health: Pola makan sehat adalah pola makan yang mengandung nutrisi seimbang untuk menjaga kesehatan tubuh.
Anda: exit
Chatbot-health: Terimakasih, Sampai jumpa!
